In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("GPU name:", torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "No GPU found")

CUDA available: True
GPU count: 1
Current device: 0
GPU name: NVIDIA GeForce RTX 3050 Laptop GPU


Logistic Regression

In [2]:
import joblib

vectorizer = joblib.load("tfidf_vectorizer_urgency.pkl")
lr_model = joblib.load("urgency_logistic_regression_email_classifier.pkl")
print("✅ Traditional ML model and vectorizer loaded successfully")

✅ Traditional ML model and vectorizer loaded successfully


In [3]:
def predict_logistic(email_text):
    X = vectorizer.transform([email_text])
    prediction = lr_model.predict(X)[0]
    confidence = lr_model.predict_proba(X).max()
    return prediction, confidence


In [4]:
email = "Please review the attached document for the project updates and let me know your feedback."

label, score = predict_logistic(email)
print("Logistic Regression Prediction:", label)
print("Confidence:", round(score, 3))


Logistic Regression Prediction: medium
Confidence: 0.998


In [4]:
email = " check for the database error asap"

label, score = predict_logistic(email)
print("Logistic Regression Prediction:", label)
print("Confidence:", round(score, 3))


Logistic Regression Prediction: high
Confidence: 1.0


In [5]:
email = "Thanks for your prompt response to my query."

label, score = predict_logistic(email)
print("Logistic Regression Prediction:", label)
print("Confidence:", round(score, 3))


Logistic Regression Prediction: low
Confidence: 0.957


DistilBERT predictions

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [8]:
tokenizer = AutoTokenizer.from_pretrained("models/urgency_distilbert_classifier")
bert_model = AutoModelForSequenceClassification.from_pretrained(
    "models/urgency_distilbert_classifier"
)
bert_model.eval()
print("✅ DistilBERT model and tokenizer loaded successfully")

✅ DistilBERT model and tokenizer loaded successfully


In [14]:
label_map = {
    0: "high_urgency",
    1: "medium_urgency",
    2: "low_urgency",
}

def predict_distilbert(email_text):
    inputs = tokenizer(
        email_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )

    with torch.no_grad():
        outputs = bert_model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)
    confidence, predicted_class = torch.max(probs, dim=1)

    return label_map[predicted_class.item()], confidence.item()



In [15]:
email = "Check for the database error"

label, score = predict_distilbert(email)
print("DistilBERT Prediction:", label)
print("Confidence:", round(score, 3))


DistilBERT Prediction: medium_urgency
Confidence: 1.0


In [16]:
email = "Check for the database error asap"

label, score = predict_distilbert(email)
print("DistilBERT Prediction:", label)
print("Confidence:", round(score, 3))


DistilBERT Prediction: high_urgency
Confidence: 1.0


In [20]:
email = "Thank you for your prompt response to my query."

label, score = predict_distilbert(email)
print("DistilBERT Prediction:", label)
print("Confidence:", round(score, 3))


DistilBERT Prediction: medium_urgency
Confidence: 1.0
